# FUSIONAR DATA RECOPILADA: MD + SOFAESCORE 

In [1]:
import pandas as pd
import openpyxl
import os 
from unidecode import unidecode
import Levenshtein 

In [2]:
def guardar_en_excell():
    # Obtener las listas de las filas
    fila_excel1 = df1.iloc[index_df1, :].tolist()
    fila_excel2 = df2.iloc[index_df2, :].tolist()

    # Concatenar las listas
    fila_concatenada = fila_excel2 + fila_excel1

    # Crear un DataFrame de pandas con una sola fila y múltiples columnas
    df_nueva_fila = pd.DataFrame([fila_concatenada])

    # Leer el archivo Excel existente
    df_existente = pd.read_excel(f'../data/{partido}.xlsx', header=None)

    # Concatenar el DataFrame existente con la nueva fila
    df_final = pd.concat([df_existente, df_nueva_fila], ignore_index=True)

    # Escribir el DataFrame final en el archivo Excel
    df_final.to_excel(f'../data/{partido}.xlsx', index=False, header=False)

In [3]:
jugadoresS_noencontrados = ["Marc-André ter Stegen", "Adria Miquel Bosch Sanchis", "Sergio Ruiz Alonso", "Abderrahman Rebbach", "Kaiky", "Alejandro Pozo", "Lázaro", "Luis Javier Suárez", "Abdessamad Ezzalzouli", "Iván Cuéllar", "Djené", "Maximiliano Gómez", "Mamadou Mbaye", "Fali", "Anthony Lozano", "José María Giménez", "Sandro Ramírez", "Reinildo Isnard Mandava", "Chimy Ávila", "Pablo Ibáñez Lumbreras", "Portu", "Juan Carlos", "José Manuel Arnáiz", "Federico Valverde", "Alfonso Espino", "Ismaila Ciss", "Josep Chavarría", "José Pozo", "Imanol García de Albéniz", "Peru Nolaskoain Esnal", "Malcom Ares"] 
jugadoresMD_noencontrados = ["Ter Stegen", "Miki Bosch", "Sergio Ruiz", "Abde Rebbach", "Kaiky Fernandes", "Álex Pozo", "Lázaro Vinicius", "Luis Suárez", "Abde Ezzalzouli", "Pichu Cuéllar", "Dakonam Djené", "Maxi Gómez", "Momo Mbaye", "Fali Giménez", "Choco Lozano", "José Giménez", "Sandro", "Reinildo Mandava", "Ezequiel Ávila", "Pablo Ibáñez", "Cristian Portu", "Juan Carlos Martín", "José Arnaiz", "Fede Valverde", "Pacha Espino", "Pathé Ciss", "Pep Chavarría", "José Ángel Pozo", "Imanol García", "Peru Nolaskoain", "Malcom Adu Ares"]

In [4]:
import pandas as pd

jornadaS = 'jornada_12'
jornadaF = 'J12'
partido ="performance_jugadores_partido_Valencia_Granada"

# Rutas de los archivos Excel
excel1_path = f'../Sofascore-data-collector/performance_jugadores_por_partidos/{jornadaS}/xlsx/{partido}.xlsx'
excel2_path = f'../Mister-Fantasy-Mundo-Deportivo-Scraper/output_scraper/{jornadaF}.xlsx'

# Leer los datos de los archivos Excel
df1 = pd.read_excel(excel1_path, header=None)
df2 = pd.read_excel(excel2_path, header=None)

# Obtener todas las celdas de la fila 1 (que ahora es la segunda fila después de desactivar el encabezado)
fila_excel1 = df1.iloc[0, :].dropna().tolist()
fila_excel2 = df2.iloc[0, :].dropna().tolist()

# Concatenar las listas
fila_concatenada =  fila_excel2 + fila_excel1

# Crear un DataFrame de pandas con una sola fila y múltiples columnas
df = pd.DataFrame([fila_concatenada])

# Escribir el DataFrame en un archivo Excel
df.to_excel(f'../data/{partido}.xlsx', index=False, header=False)

# Inicializar el conjunto de valores encontrados
valores_encontrados = set()
df_fusionado = pd.DataFrame()
encabezado=0
contador_coincidencias=0
contador_manual=0
contador_global=0

print("_____________________________________________________________________________________________________")   
print("Buscando coincidencia entre jugadores...\n")

# Iterar sobre las filas de df1 y comparar con las filas de df2
for index_df1, row_df1 in df1.iterrows():
    value_to_compare1o =row_df1.iloc[0] 
    value_to_compare1 =value_to_compare1o.lower()
    value_to_compare1 =unidecode(value_to_compare1)
    value_to_compare1 =unidecode(value_to_compare1)
    value_to_compare1=value_to_compare1.replace(" ", "")
    
    coincidencia_encontrada = False
    contador_global+=1
    # Iterar sobre las filas de df2
    for index_df2, row_df2 in df2.iterrows():
        #print("-----",value_to_compare1,"-----",value_to_compare2,"-----")
        value_to_compare2o =row_df2.iloc[0]
        value_to_compare2 =value_to_compare2o.lower()
        value_to_compare2 =unidecode(value_to_compare2)
        value_to_compare2 = unidecode(value_to_compare2)
        value_to_compare2=value_to_compare2.replace(" ", "")
        
        # Calcular la distancia de Levenshtein
        distancia = Levenshtein.distance(value_to_compare1, value_to_compare2)
        # Establecer un umbral para considerar coincidencias
        umbral = 2  

        if distancia <= umbral:
                print(f"Coincidencia encontrada: excell1-fila-{index_df1} <-> excell2-fila.{index_df2} , {value_to_compare1} == {value_to_compare2}")
                
                valores_encontrados.add(value_to_compare1) 

                guardar_en_excell()

                contador_coincidencias +=1
                coincidencia_encontrada = True
       
    # Imprimir si no se encontró ninguna coincidencia
    if not coincidencia_encontrada:
        if value_to_compare1!="nombre":
            print("------------------------------------------------------------------------------------------------")
            print(f"Coincidencia NO encontrada: excell1-fila-{index_df1} en {value_to_compare1}")
            print("------------------------------------------------------------------------------------------------")

print("_____________________________________________________________________________________________________")       
print("Buscando jugaodres manualmente que no hicieron match...\n")
for jugadorS, jugadorMD in zip(jugadoresS_noencontrados, jugadoresMD_noencontrados):
    for index_df1, row_df1 in df1.iterrows():
        value_to_compare1o = row_df1.iloc[0]
        for index_df2, row_df2 in df2.iterrows():
            value_to_compare2o = row_df2.iloc[0]

            if value_to_compare1o == jugadorS and value_to_compare2o == jugadorMD:
                print("Coincidencia encontrada: ",jugadorS)
                guardar_en_excell()
                contador_manual+=1
  
    

print("\n_____________________________________________________________________________________________________")
print("Total coincidencias: ",contador_coincidencias,"/",contador_global-1)
print("Añadidos manualmente: ",contador_manual)
print("Jugadores no disponibles en MisterFantasy:",((contador_global-1)-(contador_coincidencias+contador_manual)))
print("Precisión: ", (((contador_coincidencias+contador_manual)/(contador_global-1))*100),"%")
print("Done")

_____________________________________________________________________________________________________
Buscando coincidencia entre jugadores...

Coincidencia encontrada: excell1-fila-1 <-> excell2-fila.55 , giorgimamardashvili == giorgimamardashvili
Coincidencia encontrada: excell1-fila-2 <-> excell2-fila.110 , thierrycorreia == thierrycorreia
Coincidencia encontrada: excell1-fila-3 <-> excell2-fila.192 , cristhianmosquera == cristhianmosquera
Coincidencia encontrada: excell1-fila-4 <-> excell2-fila.214 , gabrielpaulista == gabrielpaulista
Coincidencia encontrada: excell1-fila-5 <-> excell2-fila.61 , joseluisgaya == joseluisgaya
Coincidencia encontrada: excell1-fila-6 <-> excell2-fila.74 , franperez == franperez
Coincidencia encontrada: excell1-fila-7 <-> excell2-fila.10 , pepelu == pepelu
Coincidencia encontrada: excell1-fila-8 <-> excell2-fila.19 , javierguerra == javierguerra
Coincidencia encontrada: excell1-fila-9 <-> excell2-fila.298 , selimamallah == selimamallah
Coincidencia enco